In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 163 kB in 7s (23.0 kB/s)
Reading package lists... Don

In [2]:
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-14 00:11:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.07MB/s    in 0.8s    

2022-08-14 00:11:17 (1.07 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [61]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DigitalVideoGamesETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [62]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://big-data-challenge-bucket.s3.eu-west-2.amazonaws.com/amazon_reviews_us_Digital_Video_Games_v1_00.tsv" 
spark.sparkContext.addFile(url)
video_games_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv"), sep='\t', header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")

# Show DataFrame
video_games_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-01-31 00:08:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [15]:
print(f'There are {video_games_reviews_df.count()} rows and {len(video_games_reviews_df.columns)} columns in this datatset.')

There are 145431 rows and 15 columns in this datatset.


In [70]:
# Drop unnecessary columns including "vine" as all are non vine reviews
video_games_reviews_df = video_games_reviews_df.select(["customer_id", "product_title","star_rating","helpful_votes","total_votes", "verified_purchase"])
video_games_reviews_df.show()

+-----------+--------------------+-----------+-------------+-----------+-----------------+
|customer_id|       product_title|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------------+-----------+-------------+-----------+-----------------+
|   21269168|Madden NFL 16 - X...|          2|            2|          3|                N|
|     133437| Xbox Live Gift Card|          5|            0|          0|                Y|
|   45765011|Command & Conquer...|          5|            0|          0|                Y|
|     113118|Playstation Plus ...|          5|            0|          0|                Y|
|   22151364|Saints Row IV - E...|          5|            0|          0|                Y|
|   22151364|Double Dragon: Ne...|          5|            0|          0|                Y|
|   38426028|              Sims 4|          4|            0|          0|                Y|
|    6057518|Playstation Netwo...|          5|            0|          0|                Y|

# Verified Purchase Reviews Analysis

In [42]:
# Create Verified Purchase Reviews DataFrame 
from pyspark.sql.functions import col 
verified_purchase_df = video_games_reviews_df.filter(col("verified_purchase")  == "Y")
verified_purchase_df.show()

+-----------+-----------+--------------------+-----------+-------------+-----------+-----------------+
|marketplace|customer_id|       product_title|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+-----------+--------------------+-----------+-------------+-----------+-----------------+
|         US|     133437| Xbox Live Gift Card|          5|            0|          0|                Y|
|         US|   45765011|Command & Conquer...|          5|            0|          0|                Y|
|         US|     113118|Playstation Plus ...|          5|            0|          0|                Y|
|         US|   22151364|Saints Row IV - E...|          5|            0|          0|                Y|
|         US|   22151364|Double Dragon: Ne...|          5|            0|          0|                Y|
|         US|   38426028|              Sims 4|          4|            0|          0|                Y|
|         US|    6057518|Playstation Netwo...|          5|            0| 

In [43]:
# Top 10 based on Helpful votes / Verified Purchase
from pyspark.sql.functions import desc
top_ten_helpful = verified_purchase_df.orderBy(verified_purchase_df.helpful_votes.desc())
top_ten_helpful.show(10, truncate=False)

+-----------+-----------+-------------------------+-----------+-------------+-----------+-----------------+
|marketplace|customer_id|product_title            |star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+-----------+-------------------------+-----------+-------------+-----------+-----------------+
|US         |26834793   |SimCity - Limited Edition|1          |5068         |5251       |Y                |
|US         |50432755   |SimCity - Limited Edition|5          |2509         |2974       |Y                |
|US         |12708262   |Xbox Live Subscription   |5          |993          |1077       |Y                |
|US         |34790787   |Playstation Network Card |5          |882          |959        |Y                |
|US         |14637800   |Xbox Live Subscription   |5          |623          |688        |Y                |
|US         |26834219   |Playstation Network Card |5          |546          |625        |Y                |
|US         |20952261   |Pla

In [44]:
# Top 10 based on Total Votes / Verified Purchase
top_ten_total_votes = verified_purchase_df.orderBy(verified_purchase_df.total_votes.desc())
top_ten_total_votes.show(10, truncate=False)

+-----------+-----------+----------------------------------+-----------+-------------+-----------+-----------------+
|marketplace|customer_id|product_title                     |star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+-----------+----------------------------------+-----------+-------------+-----------+-----------------+
|US         |26834793   |SimCity - Limited Edition         |1          |5068         |5251       |Y                |
|US         |50432755   |SimCity - Limited Edition         |5          |2509         |2974       |Y                |
|US         |12708262   |Xbox Live Subscription            |5          |993          |1077       |Y                |
|US         |34790787   |Playstation Network Card          |5          |882          |959        |Y                |
|US         |14637800   |Xbox Live Subscription            |5          |623          |688        |Y                |
|US         |26834219   |Playstation Network Card          |5   

In [46]:
# Top 10 Reviewed Products / Verified Purchase
top_products = verified_purchase_df.groupby("product_title")\
              .agg({"product_title": "count"})\
              .withColumnRenamed("count(product_title)", "product_count")

top_ten_products = top_products.orderBy(top_products.product_count.desc())
top_ten_products.show(10, truncate=False)

+--------------------------------------------------------------+-------------+
|product_title                                                 |product_count|
+--------------------------------------------------------------+-------------+
|Playstation Network Card                                      |13116        |
|Xbox Live Subscription                                        |6810         |
|Playstation Plus Subscription                                 |4503         |
|Xbox Live Gift Card                                           |3342         |
|Xbox 360 Live Points Card                                     |2817         |
|SimCity - Limited Edition                                     |1403         |
|Final Fantasy XIV: A Realm Reborn                             |1129         |
|Battlefield 4                                                 |1064         |
|Battlefield 3                                                 |872          |
|Command and Conquer The Ultimate Collection [Online

In [69]:
# Verified purchase reviews summary statistics table
summary_statistics = verified_purchase_df.select(["star_rating","helpful_votes","total_votes"]).describe()
summary_statistics.show()

+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|            124316|            124316|            124316|
|   mean|  3.92431384536182|1.0034026191318897|1.9605360532835676|
| stddev|1.5038404735807427| 17.35840448705645|19.514827574772877|
|    min|                 1|                 0|                 0|
|    max|                 5|              5068|              5251|
+-------+------------------+------------------+------------------+



# Unverified Purchase Reviews Analysis

In [53]:
# Create Unverified Purchase Reviews DataFrame 
from pyspark.sql.functions import col

unverified_purchase_df = video_games_reviews_df.filter(col("verified_purchase")  == "N")
unverified_purchase_df.show()

+-----------+-----------+--------------------+-----------+-------------+-----------+-----------------+
|marketplace|customer_id|       product_title|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+-----------+--------------------+-----------+-------------+-----------+-----------------+
|         US|   21269168|Madden NFL 16 - X...|          2|            2|          3|                N|
|         US|    8926809|Sid Meier's Civil...|          1|            0|          0|                N|
|         US|   42489718|Playstation Plus ...|          5|            0|          0|                N|
|         US|   16435800| Xbox Live Gift Card|          1|            0|          0|                N|
|         US|     116440|Xbox Live Subscri...|          1|            0|          1|                N|
|         US|   12551634|Playstation Netwo...|          1|            0|          0|                N|
|         US|   30665883|Kitty Powers' Mat...|          5|            1| 

In [54]:
# Top 10 based on Helpful votes / Unverified Purchase
from pyspark.sql.functions import desc
top_ten_helpful = unverified_purchase_df.orderBy(unverified_purchase_df.helpful_votes.desc())
top_ten_helpful.show(10, truncate=False)

+-----------+-----------+-----------------------------+-----------+-------------+-----------+-----------------+
|marketplace|customer_id|product_title                |star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+-----------+-----------------------------+-----------+-------------+-----------+-----------------+
|US         |37248460   |SimCity - Limited Edition    |1          |3789         |3948       |N                |
|US         |44692602   |Playstation Network Card     |5          |2384         |2541       |N                |
|US         |26458957   |SimCity - Limited Edition    |1          |1823         |1943       |N                |
|US         |13560072   |SimCity - Limited Edition    |1          |1139         |1212       |N                |
|US         |29805145   |SimCity - Limited Edition    |1          |1100         |1151       |N                |
|US         |52990130   |SimCity - Limited Edition    |1          |685          |735        |N          

In [55]:
# Top 10 based on Total Votes / Unverified Purchase
top_ten_total_votes = unverified_purchase_df.orderBy(unverified_purchase_df.total_votes.desc())
top_ten_total_votes.show(10, truncate=False)

+-----------+-----------+---------------------------------------+-----------+-------------+-----------+-----------------+
|marketplace|customer_id|product_title                          |star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+-----------+---------------------------------------+-----------+-------------+-----------+-----------------+
|US         |37248460   |SimCity - Limited Edition              |1          |3789         |3948       |N                |
|US         |44692602   |Playstation Network Card               |5          |2384         |2541       |N                |
|US         |26458957   |SimCity - Limited Edition              |1          |1823         |1943       |N                |
|US         |13560072   |SimCity - Limited Edition              |1          |1139         |1212       |N                |
|US         |29805145   |SimCity - Limited Edition              |1          |1100         |1151       |N                |
|US         |52990130   

In [56]:
# Top 10 Reviewed Products / Unverified Purchase
top_unverified_products = unverified_purchase_df.groupby("product_title")\
              .agg({"product_title": "count"})\
              .withColumnRenamed("count(product_title)", "product_count")

top_unverified_products = top_unverified_products.orderBy(top_unverified_products.product_count.desc())
top_unverified_products.show(10, truncate=False)

+---------------------------------+-------------+
|product_title                    |product_count|
+---------------------------------+-------------+
|SimCity - Limited Edition        |2018         |
|Playstation Network Card         |526          |
|Xbox Live Subscription           |497          |
|Battlefield 4                    |389          |
|Battlefield 3                    |288          |
|Xbox 360 Live Points Card        |219          |
|Playstation Plus Subscription    |209          |
|Thief Gold [Download]            |205          |
|Mass Effect 3                    |146          |
|Final Fantasy XIV: A Realm Reborn|144          |
+---------------------------------+-------------+
only showing top 10 rows



In [58]:
# Unverified purchase reviews summary statistics table
summary_unverified = unverified_purchase_df.select(["star_rating","helpful_votes","total_votes"]).describe()
summary_unverified.show()

+-------+------------------+------------------+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|
+-------+------------------+------------------+-----------------+
|  count|             21115|             21115|            21115|
|   mean|3.4340042623727207|4.3052806062041205|7.023632488752072|
| stddev|1.6774697470532463|  37.8266062931745| 41.7573626193688|
|    min|                 1|                 0|                0|
|    max|                 5|              3789|             3948|
+-------+------------------+------------------+-----------------+



In [67]:
# Get the total number of verified purchase reviews in this dataset
verified = verified_purchase_df.groupby("verified_purchase").agg({"verified_purchase":"count"})
verified.show()

+-----------------+------------------------+
|verified_purchase|count(verified_purchase)|
+-----------------+------------------------+
|                Y|                  124316|
+-----------------+------------------------+



In [68]:
# Get the total number of unverified purchase reviews in this dataset
unverified = unverified_purchase_df.groupby("verified_purchase").agg({"verified_purchase":"count"})
unverified.show()

+-----------------+------------------------+
|verified_purchase|count(verified_purchase)|
+-----------------+------------------------+
|                N|                   21115|
+-----------------+------------------------+

